# Map Visualization in Python

> Map Visualization

- toc: true 
- badges: true
- comments: true
- categories: [Map Visualization]
- image: images/sql_1.png

## 1) Import libraries

In [1]:
import json
import requests
import pandas as pd
import plotly.express as px

## 2) Data preparation 

The original dataset source: https://raw.githubusercontent.com/suchith91/wdi/master/WDI_Data_Selected.csv

### 2.1) Load dataset

In [2]:
# This line will disappear in the portfolio page
# Step 1: Import dataset
data = pd.read_csv("https://raw.githubusercontent.com/leonardodecastro/data/main/WDI_Data_Selected.csv", encoding='cp1252').drop(['Indicator Code'], axis= 1)

# Step 2: Change dataset to allow for the use of map libraries
world_data_df = pd.melt(data, id_vars=['Country Name', 'Country Code','Indicator Name'], var_name='Year', value_name='Indicator Value')
world_data_df['Year'] = world_data_df['Year'].astype('int')
world_data_df.head(2)

,Country Name,Country Code,Indicator Name,Year,Indicator Value
0,Arab World,ARB,CO2 emissions (metric tons per capita),1960,0.644
1,Arab World,ARB,Exports of goods and services (% of GDP),1960,NaN


### 2.2) Create datasets for each topic

In [3]:
# This line will disappear in the portfolio page
# Create dataset 1: CO2 emissions per year
CO2_emissions_df = world_data_df[world_data_df['Indicator Name'] == 'CO2 emissions (metric tons per capita)']

# Create dataset 2: Exports of goods per year
export_perc_GDP_df = world_data_df[world_data_df['Indicator Name'] == 'Exports of goods and services (% of GDP)']

# Create dataset 3: Forest area per year
land_use_perc_df = world_data_df[world_data_df['Indicator Name'] == 'Forest area (% of land area)']

# Create dataset 4: GDP Growth per year
GDP_growth_df = world_data_df[world_data_df['Indicator Name'] == 'GDP growth (annual %)']

# Create dataset 5: Imports of goods per year
import_perc_GDP_df = world_data_df[world_data_df['Indicator Name'] == 'Imports of goods and services (% of GDP)']

# Create dataset 6: Poverty headcount per year
poverty_perc_pop_df = world_data_df[world_data_df['Indicator Name'] == 'Poverty headcount ratio at national poverty lines (% of population)']

# Create dataset 7: Unemployment per year
unemployment_perc_df = world_data_df[world_data_df['Indicator Name'] == 'Unemployment, total (% of total labor force) (national estimate)']

# Create dataset 8: Youth literacy per year
youth_literacy_perc_df = world_data_df[world_data_df['Indicator Name'] == 'Youth literacy rate, population 15-24 years, both sexes (%)']
youth_literacy_perc_df.head(2)

,Country Name,Country Code,Indicator Name,Year,Indicator Value
7,Arab World,ARB,"Youth literacy rate, population 15-24 years, b...",1960,NaN
15,Caribbean small states,CSS,"Youth literacy rate, population 15-24 years, b...",1960,NaN


### 2.3) Create value ranges for certain visualizations

In [4]:
# This line will disappear in the portfolio page


# Create value ranges for dataset 3: GDP Growth per year
bins= [GDP_growth_df['Indicator Value'].min()-1, -10, -5, 0, 5, 10, GDP_growth_df['Indicator Value'].max()+1]
labels = ['< -10 %' , '-10% to -5%', '-5% to 0%','0% to 5%','5% to 10%','> 10%']
GDP_growth_df.loc[:,'Growth Ranges']= pd.cut(GDP_growth_df['Indicator Value'], bins=bins, labels=labels, right=False).astype('str')

# Create value ranges for dataset 7: Unemployment per year
bins= [unemployment_perc_df['Indicator Value'].min()-1, 5, 10, 15, 20, unemployment_perc_df['Indicator Value'].max()+1]
labels = ['0% - 5%' , '5% - 10%', '10% - 15%','15% - 20%','> 20%']
unemployment_perc_df.loc[:,'Unemployment Ranges']= pd.cut(unemployment_perc_df['Indicator Value'], bins=bins, labels=labels, right=False).astype('str')

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


## 3) Create Time Series Visualizations

### 3.1) Maps with Folium

### 3.2) Maps with Plotly

#### 3.2.1) Less beautiful map that runs fast

We need to use ISO codes with 3 letters for plotly.express to work properly

In [5]:
# This line will disappear in the portfolio page
# Create visualization
fig = px.choropleth(GDP_growth_df,
                    locations="Country Code",        # Column where country code with 3 letters can be found
                    color="Indicator Value",         # Indicator Value is the numerical value we want to examine
                    hover_name="Country Code",       # Column to add to hover information
                    animation_frame='Year',          # Show column that will be used in the animation frame
                    title='GDP Yealy Growth (%)',
                    color_continuous_scale=px.colors.diverging.RdYlGn)

# Control the speed of the transitions
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 10
fig.show()

#### 3.2.2) Very beautiful map that takes time to run

> Since the next visualization takes time to run, we will limit to 25 years worth of data instead of 65 years.

The original file "countries_world.json" was simplified by around 50% using an online tool available at https://mapshaper.org/. This was necessary since this type of file is somewhat large. If we were dealing with the visualization of a single year, we could use the original file instead of the simplified one. 

In [6]:
# Import json file
geo_world  = requests.get("https://raw.githubusercontent.com/leonardodecastro/data/main/countries_world_simplified.json").json()

# Create image
fig = px.choropleth_mapbox(GDP_growth_df[GDP_growth_df['Year']>=1990],                                     # Dataframe with the information
                           geojson=geo_world,                             # Geojson with ISO country codes as IDs
                           locations = "Country Code",                        # Define the limits on the map/geography
                           color = 'Indicator Value',                         # defining the color of the scale through the database
                           hover_name = "Country Code",                       # the information in the box
                           hover_data =["Country Code","Indicator Value"],
                           featureidkey="properties.ADM0_A3",
                           mapbox_style = "carto-positron",                   #defining a new map style 
                           center={"lat":10, "lon": 5},                    #define the limits that will be plotted
                           zoom = 1,                                          #map view size
                           opacity = 0.5,                                     #opacity of the map color, to appear the background
                           animation_frame = 'Year',
                           title='GDP Yealy Growth (%)',
                           height = 600,
                           width = 1000,
                           color_continuous_scale=px.colors.diverging.RdYlGn) #creating the application based on the year

# Adjust map layout styling
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

# Control the speed of the transitions
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 2000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 20
fig.show()

Output hidden; open in https://colab.research.google.com to view.

#### 3.2.3) Very beautiful map that takes even more time to run

This visualization is more computationally intense than the previous one since we specify the discrete colormap with 6 labels.

In [7]:
# Create image
fig = px.choropleth_mapbox(GDP_growth_df[GDP_growth_df['Year']>=2012],                                     # Dataframe with the information
                           geojson=geo_world,                             # Geojson with ISO country codes as IDs
                           locations = "Country Code",                        # Define the limits on the map/geography
                           color = 'Growth Ranges',                         # defining the color of the scale through the database
                           color_discrete_map = {'nan': '#4d4d4d', '< -10 %': '#d73027', '-10% to -5%' : '#fc8d59', '-5% to 0%' : '#fee08b',
                                                 '0% to 5%' : '#d9ef8b', '5% to 10%' : '#91cf60', '> 10%' : '#1a9850'},
                           category_orders={'Growth Ranges' : ['nan', '< -10 %' , '-10% to -5%', '-5% to 0%','0% to 5%','5% to 10%', '> 10%']},
                           hover_name = "Country Code",                       # the information in the box
                           hover_data =["Country Code","Indicator Value"],
                           featureidkey="properties.ADM0_A3",
                           mapbox_style = "carto-positron",                   #defining a new map style 
                           center={"lat":10, "lon": 5},                    #define the limits that will be plotted
                           zoom = 1,                                          #map view size
                           opacity = 0.5,                                     #opacity of the map color, to appear the background
                           animation_frame = 'Year',
                           title='GDP Yealy Growth (%)',
                           height = 600,
                           width = 1000) #creating the application based on the year

# Adjust map layout styling
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

# Control the speed of the transitions
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 2000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 20
fig.show()

Output hidden; open in https://colab.research.google.com to view.

## 4) Create Side-by-Side Visualizations

$\color{red}{\text{Change descriptions of what I am doing to adapt them for my case. Remember to turn the last analysis into a discrete one. Use the following website: }}$ https://mahshadn.medium.com/animated-choropleth-map-with-discrete-colors-using-python-and-plotly-styling-5e208e5b6bf8

In [8]:
import ipywidgets as widgets
from plotly.subplots import make_subplots
from ipywidgets import interact, interact_manual

In [9]:
min_year = GDP_growth_df['Year'].min()

In [10]:
max_year = GDP_growth_df['Year'].max()

In [11]:
@interact
def precipitação(year=(min_year,max_year + 1)):
    
  fig_t1 = px.choropleth_mapbox(GDP_growth_df[GDP_growth_df['Year']==year],                                     # Dataframe with the information
                            geojson=geo_world,                             # Geojson with ISO country codes as IDs
                            locations = "Country Code",                        # Define the limits on the map/geography
                            color = 'Indicator Value',                         # defining the color of the scale through the database
                            hover_name = "Country Code",                       # the information in the box
                            hover_data =["Country Code","Indicator Value"],
                            featureidkey="properties.ADM0_A3",
                            mapbox_style = "carto-positron",                   #defining a new map style 
                            opacity = 0.5,                                     #opacity of the map color, to appear the background
                            animation_frame = 'Year') #creating the application based on the year

  fig_t2 = px.choropleth_mapbox(GDP_growth_df[GDP_growth_df['Year']==year],                                     # Dataframe with the information
                            geojson=geo_world,                             # Geojson with ISO country codes as IDs
                            locations = "Country Code",                        # Define the limits on the map/geography
                            color = 'Indicator Value',                         # defining the color of the scale through the database
                            hover_name = "Country Code",                       # the information in the box
                            hover_data =["Country Code","Indicator Value"],
                            featureidkey="properties.ADM0_A3",
                            mapbox_style = "carto-positron",                   #defining a new map style 
                            opacity = 0.5,                                     #opacity of the map color, to appear the background
                            animation_frame = 'Year') 

  fig = make_subplots(rows=1, cols=2, subplot_titles=['Premier tour', 'Second tour'], specs=[[{'type': 'choroplethmapbox'}, {'type': 'choroplethmapbox'}]])

  fig.add_trace(fig_t1['data'][0], row=1, col=1)
  fig.add_trace(fig_t2['data'][0], row=1, col=2)

  fig.update_layout(coloraxis_showscale=True) # update

  fig.update_mapboxes(style='white-bg', center={'lat': 10, 'lon': 5}, zoom= 0.2)
  fig.update_layout(title_text='Élection Présidentielle 2022 - Vainqueur par département', margin={'l': 0, 'r': 0, 't': 100, 'b': 0}, height=500, width = 1200)

  return fig.show()

interactive(children=(IntSlider(value=1988, description='year', max=2016, min=1960), Output()), _dom_classes=(…